In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/lucinnnal/kaggle_cnn.git

Cloning into 'kaggle_cnn'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 45 (delta 23), reused 35 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 27.93 KiB | 13.96 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
import os
os.chdir('kaggle_cnn')

In [2]:
!cp /content/drive/MyDrive/data.zip /content/
!unzip /content/data.zip -d /content/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/data 복사본/train/5_05969.jpg  
  inflating: /content/data 복사본/train/2_03367.jpg  
  inflating: /content/data 복사본/train/8_08269.jpg  
  inflating: /content/data 복사본/train/5_06253.jpg  
  inflating: /content/data 복사본/train/8_08282.jpg  
  inflating: /content/data 복사본/train/4_05315.jpg  
  inflating: /content/data 복사본/train/9_08801.jpg  
  inflating: /content/data 복사본/train/2_02938.jpg  
  inflating: /content/data 복사본/train/4_05301.jpg  
  inflating: /content/data 복사본/train/4_04779.jpg  
  inflating: /content/data 복사본/train/5_05996.jpg  
  inflating: /content/data 복사본/train/0_09972.jpg  
  inflating: /content/data 복사본/train/5_06247.jpg  
  inflating: /content/data 복사본/train/2_03398.jpg  
  inflating: /content/data 복사본/train/4_04989.jpg  
  inflating: /content/data 복사본/train/5_05766.jpg  
  inflating: /content/data 복사본/train/10_01189.jpg  
  inflating

In [ ]:
!python baseline.py

wandb: Currently logged in as: kipyo39 (kipyo39-sungkyunkwan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.11
wandb: Run data is saved locally in /content/kaggle_cnn/wandb/run-20250518_232320-2xdq968e
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run food_classification_improved
wandb: ⭐️ View project at https://wandb.ai/kipyo39-sungkyunkwan-university/food_classification
wandb: 🚀 View run at https://wandb.ai/kipyo39-sungkyunkwan-university/food_classification/runs/2xdq968e
One /content/data/train sample /content/data/train/0_00000.jpg
One /content/data/validation sample /content/data/validation/0_00003.jpg
Using device: cuda
Epoch 1/200 [Train]: 100% 125/125 [01:00<00:00,  2.06it/s, loss=2.23, acc=0.22]
[ Train | 001/200 ] loss = 2.23001, acc = 0.22007
Epoch 1/200 [Valid]: 100% 42/42 [00:14<00:00,  2.88it/s, loss=2.21, acc=0.288]
[ Valid | 001/200 ] loss = 2.20551, acc = 0.28786
Best model

In [3]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# Set experiment name and wandb project
_exp_name = "food_classification_improved"
project_name = "food_classification"

# Test/validation transformations
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files=None, is_test=False):
        super(FoodDataset).__init__()
        self.path = path
        self.is_test = is_test
        self.files = sorted([os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files is not None:
            self.files = files
        print(f"One {path} sample", self.files[0])
        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname).convert('RGB')  # Ensure RGB format
        im = self.transform(im)
        file_id = os.path.basename(fname).split(".")[0]  # Extract the ID from the filename
        if self.is_test:
            return im, -1, file_id  # Return file_id only for test set
        else:
            label = int(fname.split("/")[-1].split("_")[0])
            return im, label

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=11):
        super(ResNet18, self).__init__()
        self.in_planes = 32  # 채널 수를 줄임

        # Initial convolution layer for 128x128 input
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),  # [32, 128, 128]
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # [32, 64, 64]
        )

        # ResNet stages with BasicBlock - [2,2,2,2] configuration
        self.layer1 = self._make_layer(BasicBlock, 32, 2, stride=1)    # [32, 64, 64]
        self.layer2 = self._make_layer(BasicBlock, 64, 2, stride=2)    # [64, 32, 32]
        self.layer3 = self._make_layer(BasicBlock, 128, 2, stride=2)   # [128, 16, 16]
        self.layer4 = self._make_layer(BasicBlock, 256, 2, stride=2)   # [256, 8, 8]

        # Global average pooling and classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(256 * BasicBlock.expansion, num_classes)
        )

        self._initialize_weights()

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)      # [B, 32, 64, 64]

        x = self.layer1(x)     # [B, 32, 64, 64]
        x = self.layer2(x)     # [B, 64, 32, 32]
        x = self.layer3(x)     # [B, 128, 16, 16]
        x = self.layer4(x)     # [B, 256, 8, 8]

        x = self.avgpool(x)    # [B, 256, 1, 1]
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [4]:
def test_prediction():
    _dataset_dir = "/content/data/"
    _exp_name = "food_classification_improved"
    batch_size = 64
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load test dataset
    test_set = FoodDataset(os.path.join(_dataset_dir, "test"), tfm=test_tfm, is_test=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Load best model
    model_best = ResNet18(num_classes=11).to(device)
    checkpoint = torch.load("/content/food_classification_improved_best (14).ckpt")
    model_best.load_state_dict(checkpoint['model_state_dict'])
    model_best.eval()

    # Prediction with tqdm progress bar
    prediction = []
    file_ids = []

    with torch.no_grad():
        for data, _, file_id in tqdm(test_loader, desc="Generating predictions"):
            test_pred = model_best(data.to(device))
            test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
            prediction += test_label.squeeze().tolist()
            file_ids += file_id  # Append original IDs

    # Create submission CSV
    df = pd.DataFrame()
    df["ID"] = file_ids
    df["Category"] = prediction
    df.to_csv("submission.csv", index=False)
    print("Submission file created successfully!")

In [5]:
test_prediction()

One /content/data/test sample /content/data/test/00002.jpg


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Generating predictions:   0%|          | 0/42 [00:00<?, ?it/s]

Submission file created successfully!
